In [134]:
from sklearn import datasets
import pandas as pd
#datasets.load*?

In [135]:
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

In [136]:
# df = sklearn_to_df(datasets.load_iris())
# X = df.drop('target',axis = 'columns')
# y = df['target']
# cat_cols = []
# num_cols = X.columns
#X,y = datasets.make_regression(n_samples= 1000,n_features=10,noise = 0.2)

In [137]:
df = pd.read_csv('Data - Parkinsons')

#df[['id_0','id_1','id_2','id_3']] = df['name'].str.split('_',expand = True)

#df = df.drop('name', axis = 1)

X = df.drop('status',axis = 'columns')
y = df['status']

cat_cols = df.select_dtypes(include = ['object']).columns
num_cols = df.select_dtypes(exclude = ['object']).drop('status',axis = 1).columns

In [138]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size = 0.3, random_state=1)

In [139]:
from sklearn import set_config
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler,PolynomialFeatures
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import cross_val_score
set_config(display='diagram')

num_proc = make_pipeline(SimpleImputer(strategy='median'), StandardScaler(),
                         PolynomialFeatures(degree = 1,interaction_only=True))

cat_proc = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(handle_unknown='ignore'))


preprocessor = make_column_transformer((num_proc, num_cols),
                                       (cat_proc, cat_cols))


model = [LinearRegression()]
pipe = make_pipeline(preprocessor,model[0])
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('polynomialfeatures',
                                                                   PolynomialFeatures(degree=1,
                                                                                      interaction_only=True))]),
                                                  Index(['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', '...
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'RPDE', 'DFA', 'spread1',
       'spread2', 'D2', 'PPE'],
      dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['name'], dtype='object'))])),
                ('linearregression', LinearRegression())])

In [140]:
from sklearn import ensemble, neighbors,linear_model,svm

model_list = [#regressors
                linear_model.LinearRegression(),linear_model.Lasso(),linear_model.Ridge(),
              
              #ensembling - regressors
                ensemble.RandomForestRegressor(max_depth = 3),ensemble.AdaBoostRegressor(),ensemble.GradientBoostingRegressor(),
    
              #classifiers
                linear_model.LogisticRegression(max_iter = 500), neighbors.KNeighborsClassifier(),svm.SVC(),             
    
              #ensembling - classifiers
                ensemble.RandomForestClassifier(n_estimators=500,max_depth = 3),ensemble.AdaBoostClassifier(learning_rate=0.5),ensemble.GradientBoostingClassifier()

              ]

In [141]:
rejected_models = []
results = []
print('{:30s}Train  /  Test  /  Cross Validation'.format('Model'))
print('-----------------------------------------------------------')
for model in model_list:
    pipe = make_pipeline(preprocessor,model)
    #pipe = make_pipeline(model)
    try:        
        pipe.fit(X_train,y_train)
        
        model_name = type(model).__name__
        train_results = np.abs(pipe.score(X_train,y_train))
        test_results = np.abs(pipe.score(X_test,y_test))
        cross_val_scores = np.abs(cross_val_score(pipe, X, y, cv=5)) 
        cval_score = cross_val_scores.mean()

        results.append([model_name, train_results, test_results,cval_score])
        print('{:30s}{:.4f} / {:.4f}  /  {:.4f} '.format(model_name, train_results, test_results,cval_score))

    except:
        rejected_models.append(type(model).__name__)

df_results = pd.DataFrame(results, columns=["Model", "Train Results","Test Results",'Test Cross Validation Results'])

Model                         Train  /  Test  /  Cross Validation
-----------------------------------------------------------
LinearRegression              1.0000 / 0.3811  /  0.3497 
Lasso                         0.0000 / 0.0542  /  0.2588 
Ridge                         0.8693 / 0.3802  /  0.3413 
RandomForestRegressor         0.8428 / 0.4848  /  0.2798 
AdaBoostRegressor             0.9960 / 0.7538  /  0.3642 
GradientBoostingRegressor     0.9992 / 0.6381  /  0.2477 
LogisticRegression            0.9265 / 0.8136  /  0.8103 
KNeighborsClassifier          0.9632 / 0.8644  /  0.7795 
SVC                           0.9044 / 0.7966  /  0.8462 
RandomForestClassifier        0.9118 / 0.7966  /  0.8103 
AdaBoostClassifier            1.0000 / 0.9153  /  0.7538 
GradientBoostingClassifier    1.0000 / 0.8644  /  0.7949 


In [142]:
df_results.sort_values(by = 'Test Cross Validation Results',ascending = False)

,Model,Train Results,Test Results,Test Cross Validation Results
8,SVC,0.904412,0.796610,0.846154
6,LogisticRegression,0.926471,0.813559,0.810256
9,RandomForestClassifier,0.911765,0.796610,0.810256
11,GradientBoostingClassifier,1.000000,0.864407,0.794872
7,KNeighborsClassifier,0.963235,0.864407,0.779487
10,AdaBoostClassifier,1.000000,0.915254,0.753846
4,AdaBoostRegressor,0.996037,0.753827,0.364226
0,LinearRegression,1.000000,0.381077,0.349665
2,Ridge,0.869261,0.380219,0.341250
3,RandomForestRegressor,0.842827,0.484810,0.279805


In [143]:
model = LogisticRegression()
pipe = make_pipeline(preprocessor,model)

In [132]:
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

Results = pd.DataFrame()

for train_index, test_index in kf.split(df):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = LogisticRegression()
    pipe = make_pipeline(preprocessor,model)
    pipe.fit(X_train,y_train)
    y_pred = pipe.predict(X_test)
    
    temp_df = pd.DataFrame()
    temp_df = pd.concat([X_test,y_test],axis = 1)
    temp_df['y_pred'] = y_pred
    Results = Results.append(temp_df)    

In [133]:
#Results.to_csv('Results.csv')